### Association rules by Top 5 popular Departments 
#### Department Produce

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')

In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)
# train_data['order_id'] = train_data['order_id'].astype('str')
# combine_train_prior['order_id'] = combine_train_prior['order_id'].astype('str')
# combine_train_prior['product_name'] = combine_train_prior['product_name'].astype('str')

In [194]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [ ]:
combine_train_prior

In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Produce
dept_produce=combine_train_prior[combine_train_prior['department'] =="produce"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [195]:
temp_df['order_id'].value_counts()

598905     51
128382     48
62786      43
570521     41
2022893    40
2172544    39
3411224    39
2409688    38
1142413    38
951047     37
1433824    37
1564244    37
3250288    36
1815908    36
2262504    35
169438     35
903110     35
3142604    35
2919291    35
1625962    35
119646     34
3363891    33
1517003    33
1730767    33
2584728    33
2905046    33
454332     33
3081363    33
1657484    33
3312811    33
           ..
2397063     1
43161       1
2462631     1
2479023     1
18565       1
2454459     1
928073      1
2978491     1
2962099     1
1010017     1
1526373     1
1534561     1
1485401     1
3379559     1
2634259     1
2626071     1
1337857     1
756221      1
2716219     1
731625      1
690653      1
2814539     1
707029      1
2773599     1
649649      1
592301      1
2863715     1
526733      1
3003055     1
2413384     1
Name: order_id, Length: 2506247, dtype: int64

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [ ]:
grouped_data

In [10]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
#relace NAN with 0
table1=table1.fillna(0)

In [13]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1
basket_sets = table1.applymap(encode_units)

In [40]:
## Running Apriori to get frequent itemset with a minimum suppirt of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [81]:
frequent_itemsets

,support,itemsets
0,0.058135,[Apple Honeycrisp Organic]
1,0.051078,[Asparagus]
2,0.011444,[Asparation/Broccolini/Baby Broccoli]
3,0.015637,[Baby Spinach]
4,0.215105,[Bag of Organic Bananas]
5,0.256616,[Banana]
6,0.025073,[Bartlett Pears]
7,0.014093,[Blackberries]
8,0.032335,[Broccoli Crown]
9,0.016331,[Brussels Sprouts]


In [164]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

In [165]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Large Lemon),(Limes),0.112539,0.106700,0.024059,0.213787,2.003634,0.012052,1.136207
1,(Limes),(Large Lemon),0.106700,0.112539,0.024059,0.225487,2.003634,0.012052,1.145831
2,"(Organic Strawberries, Organic Baby Spinach)",(Bag of Organic Bananas),0.033559,0.215105,0.010295,0.306763,1.426107,0.003076,1.132217
3,"(Organic Strawberries, Bag of Organic Bananas)",(Organic Baby Spinach),0.050377,0.169815,0.010295,0.204351,1.203375,0.001740,1.043406
4,"(Organic Baby Spinach, Bag of Organic Bananas)",(Organic Strawberries),0.041967,0.176931,0.010295,0.245300,1.386415,0.002869,1.090591
5,(Organic Strawberries),"(Organic Baby Spinach, Bag of Organic Bananas)",0.176931,0.041967,0.010295,0.058184,1.386415,0.002869,1.017219
6,(Organic Baby Spinach),"(Organic Strawberries, Bag of Organic Bananas)",0.169815,0.050377,0.010295,0.060622,1.203375,0.001740,1.010906
7,(Bag of Organic Bananas),"(Organic Strawberries, Organic Baby Spinach)",0.215105,0.033559,0.010295,0.047858,1.426107,0.003076,1.015018
8,(Organic Blueberries),(Bag of Organic Bananas),0.066790,0.215105,0.017899,0.267996,1.245885,0.003533,1.072255
9,(Bag of Organic Bananas),(Organic Blueberries),0.215105,0.066790,0.017899,0.083213,1.245885,0.003533,1.017913


In [166]:
## Since, ideally Lift must greater than , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Large Lemon),(Limes),0.112539,0.106700,0.024059,0.213787,2.003634,0.012052,1.136207
1,(Limes),(Large Lemon),0.106700,0.112539,0.024059,0.225487,2.003634,0.012052,1.145831
2,"(Organic Strawberries, Organic Baby Spinach)",(Bag of Organic Bananas),0.033559,0.215105,0.010295,0.306763,1.426107,0.003076,1.132217
3,"(Organic Strawberries, Bag of Organic Bananas)",(Organic Baby Spinach),0.050377,0.169815,0.010295,0.204351,1.203375,0.001740,1.043406
4,"(Organic Baby Spinach, Bag of Organic Bananas)",(Organic Strawberries),0.041967,0.176931,0.010295,0.245300,1.386415,0.002869,1.090591
5,(Organic Strawberries),"(Organic Baby Spinach, Bag of Organic Bananas)",0.176931,0.041967,0.010295,0.058184,1.386415,0.002869,1.017219
6,(Organic Baby Spinach),"(Organic Strawberries, Bag of Organic Bananas)",0.169815,0.050377,0.010295,0.060622,1.203375,0.001740,1.010906
7,(Bag of Organic Bananas),"(Organic Strawberries, Organic Baby Spinach)",0.215105,0.033559,0.010295,0.047858,1.426107,0.003076,1.015018
8,(Organic Blueberries),(Bag of Organic Bananas),0.066790,0.215105,0.017899,0.267996,1.245885,0.003533,1.072255
9,(Bag of Organic Bananas),(Organic Blueberries),0.215105,0.066790,0.017899,0.083213,1.245885,0.003533,1.017913


In [182]:
final_result.to_csv("Department_ProduceApriori.csv",sep=',')

In [183]:
test1=pd.read_csv("Department_ProduceApriori.csv")

In [184]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [185]:
test1=test1.replace({"u'": ''}, regex=True)

In [186]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [187]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [190]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [192]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_ProduceApriori.csv")

In [193]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Large Lemon,Limes,0.112539,0.106700,0.024059,0.213787,2.003634,0.012052,1.136207
1,Limes,Large Lemon,0.106700,0.112539,0.024059,0.225487,2.003634,0.012052,1.145831
2,"Organic Strawberries, Organic Baby Spinach",Bag of Organic Bananas,0.033559,0.215105,0.010295,0.306763,1.426107,0.003076,1.132217
3,"Organic Strawberries, Bag of Organic Bananas",Organic Baby Spinach,0.050377,0.169815,0.010295,0.204351,1.203375,0.001740,1.043406
4,"Organic Baby Spinach, Bag of Organic Bananas",Organic Strawberries,0.041967,0.176931,0.010295,0.245300,1.386415,0.002869,1.090591
5,Organic Strawberries,"Organic Baby Spinach, Bag of Organic Bananas",0.176931,0.041967,0.010295,0.058184,1.386415,0.002869,1.017219
6,Organic Baby Spinach,"Organic Strawberries, Bag of Organic Bananas",0.169815,0.050377,0.010295,0.060622,1.203375,0.001740,1.010906
7,Bag of Organic Bananas,"Organic Strawberries, Organic Baby Spinach",0.215105,0.033559,0.010295,0.047858,1.426107,0.003076,1.015018
8,Organic Blueberries,Bag of Organic Bananas,0.066790,0.215105,0.017899,0.267996,1.245885,0.003533,1.072255
9,Bag of Organic Bananas,Organic Blueberries,0.215105,0.066790,0.017899,0.083213,1.245885,0.003533,1.017913


In [196]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()

In [197]:
t

,index,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,2,"Organic Strawberries, Organic Baby Spinach",Bag of Organic Bananas,0.033559,0.215105,0.010295,0.306763,1.426107,0.003076,1.132217
